In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

import analysis.plotting as pl
import modules.iterable_funcs as itf

In [2]:
# Info about the variables
file = '2014-12-03_P005_Post_002'
frame = 90

In [3]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


### Load variables

In [4]:
load_dir = os.path.join('..', 'data', 'saved_variables')

population = np.load(os.path.join(load_dir, 'population.npy'))
labels = np.load(os.path.join(load_dir, 'labels.npy'))
path_matrix = np.load(os.path.join(load_dir, 'path_matrix.npy'))

In [5]:
part_types = ['Head', 'Hip', 'Thigh', 'Knee', 'Calf', 'Foot']

## Customize font

In [6]:
plt.rc('text', usetex=True)

font = {'family' : 'serif',
        'weight' : 'bold',
        'size'   : 12}

plt.rc('font', **font)  # pass in the font dict as kwargs

### Plot population

In [7]:
fig = plt.figure()

pl.scatter_labels(population, labels)

plt.legend(part_types)
plt.xlim((-150, 0))
plt.xlabel('X')
plt.ylabel('Y', rotation=0)
plt.show()

In [8]:
# fig.savefig('labelled_points.pdf', format='pdf', dpi=1200)

### Plot spheres

In [9]:
path_0, path_1 = path_matrix[3, :], path_matrix[1, :]
pop_0, pop_1 = population[path_0, :], population[path_1, :]

pl.scatter_labels(population, labels)
pl.scatter2(pop_0, s=1e3, facecolors='none', edgecolors='k')
pl.scatter2(pop_1, s=1e3, facecolors='none', edgecolors='k')

plt.legend(part_types)
plt.xlim((-150, 0))
plt.xlabel('X')
plt.ylabel('Y', rotation=0)
plt.show()

### Plot body graph

In [10]:
def generate_points(n_points_per_set):
    
    n_sets = len(n_points_per_set)

    for i, n_points in enumerate(n_points_per_set):

        y_vals = (n_sets - i) * np.ones(n_points)
        x_vals = np.linspace(0, 1, n_points + 1, endpoint=False)[1:]
        
        points = np.stack([(x, y) for x, y in zip(x_vals, y_vals)])
        
        yield points

In [11]:
n_points_per_set = [2, 3, 5, 2, 4, 5]

point_sets = [*generate_points(n_points_per_set)]

In [12]:
fig = plt.figure()

for points_a, points_b in itf.pairwise(point_sets):
        
    points = np.vstack([points_a, points_b])

    pl.scatter2(points, c='k', s=50)
    pl.connect_two_sets(points_a, points_b, c='k')
    
    
# Highlight path in red
prev_path_point = []
for row_points in point_sets:
    
    path_point = row_points[np.random.randint(row_points.shape[0])]

    pl.scatter2(path_point, c='r', s=100, zorder=3)
    
    if len(prev_path_point) > 0:
        pl.connect_points(prev_path_point, path_point, c='r', zorder=3)
    
    prev_path_point = path_point


# Label rows with body part names  
y_coords = [points[0, 1] for points in point_sets]

for i, part_type in enumerate(part_types):
    
    plt.text(1, y_coords[i]-0.075, part_type, fontsize=12)

plt.xlim((0, 1.1))
plt.axis('off')
plt.show()


In [14]:
# fig.savefig('body_graph.pdf', format='pdf', dpi=1200)